In [1]:
from transformers import TFBertTokenizer, TFBertModel
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers, metrics
import pandas as pd
import numpy as np
import tensorflow as tf

/home/yu/kaggle-tweet-sentiment/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-28 15:30:03.867741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738096203.912444    9693 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738096203.924504    9693 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 15:30:03.999128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

In [2]:
train = pd.read_csv("data1/train.csv").dropna().iloc[:10000]
submission = pd.read_csv("data1/test.csv").dropna()
print(train.info())
print(submission.info())
# TODO see if selected_text needs to be fixed to word boundaries

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         10000 non-null  object
 1   text           10000 non-null  object
 2   selected_text  10000 non-null  object
 3   sentiment      10000 non-null  object
dtypes: object(4)
memory usage: 390.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     3534 non-null   object
 1   text       3534 non-null   object
 2   sentiment  3534 non-null   object
dtypes: object(3)
memory usage: 83.0+ KB
None


In [3]:
max_text_len = 128
span_indices_size = max_text_len * (max_text_len + 1) // 2
span_indices = np.zeros((max_text_len, max_text_len), dtype=int)
span_indices[np.triu_indices(max_text_len, m=max_text_len)] = np.arange(span_indices_size)
span_indices = np.flip(span_indices, axis=1)
tokenizer = TFBertTokenizer.from_pretrained("bert-base-cased")

def tokenize(texts):
    return tokenizer(texts, padding="max_length", max_length=max_text_len)

def detokenize(ids, skip_special=True):
    return tokenizer.batch_decode(ids, skip_special_tokens=skip_special)

# https://stackoverflow.com/a/7100681
def rolling_window(a, size):
    shape = a.shape[:-1] + (a.shape[-1] - size + 1, size)
    strides = a.strides + (a. strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def find_span(texts, spans):
    texts = np.array(texts)
    spans = np.array(spans)[:, 1:]
    sizes = spans.argmin(axis=1) - 1
    span_onehot = np.zeros((texts.shape[0], span_indices_size))
    for i, text in enumerate(texts):
        window_size = sizes[i]
        matched_window = rolling_window(texts[i], window_size) == spans[i][:window_size]
        window_from = matched_window.all(axis=1).argmax()
        window_to = window_size - 1
        try:
            span_onehot[i, span_indices[window_from, window_to]] = 1
        except IndexError as e:
            print(window_from, window_to, span_indices[window_from, window_to])
            raise e
    return span_onehot

def find_y(texts, selected_texts):
    return find_span(texts["input_ids"], selected_texts["input_ids"])

def span_range(index):
    [d1], [d2] = np.where(span_indices == index)
    return d1, d1 + d2 + 1

def print_mem_info():
    info = tf.config.experimental.get_memory_info("/gpu:0")
    print(f"GPU memory usage {(info["current"] / info["peak"]):.2f}.")

tokens_train = tokenize(train.text.to_list())
selected_tokens_train = tokenize(train.selected_text.to_list())
y_train = find_y(tokens_train, selected_tokens_train)

tokenizer = None
print_mem_info()

2025-01-28 15:30:24.419327: I tensorflow_text/core/kernels/fast_bert_normalizer_model_builder.cc:211] CharacterSet built (lower_case_nfd_strip_accents=0). Trie data size (int32): 512. Normalized string pool size (byte): 1
I0000 00:00:1738096224.821500    9693 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21904 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


GPU memory usage 0.00.


In [4]:
batch_size = 2000
encoder = TFBertModel.from_pretrained("bert-base-cased")

def batch_inference(tokens, one=False):
    total = tokens["input_ids"].shape[0]
    stack = []
    while batch_size * len(stack) < total:
        start, end = min(batch_size, total - batch_size * len(stack)) * np.array([0, 1]) + batch_size * len(stack)
        batch = {k: v[start:end] for k, v in tokens.items()}
        outputs = encoder(batch).last_hidden_state
        with tf.device("/cpu:0"):
            stack.append(tf.identity(outputs))
        if one:
            break
    return tf.concat(stack, 0)

contexts_train = batch_inference(tokens_train)
encoder = None
print_mem_info()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

GPU memory usage 0.24.


In [5]:
contexts_train.shape

TensorShape([10000, 128, 768])

In [6]:
# tf.data.Dataset.from_tensors(contexts_train).save("contexts")

In [11]:
inputs = keras.Input(shape=(max_text_len, 768))
softmax = layers.Dense(span_indices_size, activation="softmax")(
          layers.Dense(128, activation="relu")(
          layers.Flatten()(
          inputs)))
ffn = keras.Model(inputs=inputs, outputs=softmax)
ffn.compile(
    optimizer=optimizers.Adam(),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
)
history = ffn.fit(contexts_train, y_train, batch_size=16, epochs=20, validation_split=0.2)

2025-01-28 15:34:51.718036: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3145728000 exceeds 10% of free system memory.


Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - categorical_accuracy: 0.1347 - categorical_crossentropy: 5.6634 - loss: 5.6634 - val_categorical_accuracy: 0.1855 - val_categorical_crossentropy: 4.7566 - val_loss: 4.7566
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - categorical_accuracy: 0.2659 - categorical_crossentropy: 4.0260 - loss: 4.0260 - val_categorical_accuracy: 0.2755 - val_categorical_crossentropy: 4.5840 - val_loss: 4.5840
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - categorical_accuracy: 0.3336 - categorical_crossentropy: 3.5028 - loss: 3.5028 - val_categorical_accuracy: 0.2725 - val_categorical_crossentropy: 4.6635 - val_loss: 4.6635
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - categorical_accuracy: 0.3784 - categorical_crossentropy: 3.1597 - loss: 3.1597 - val_categorical_accuracy: 0.2765 - val_categorical_crossentropy: 4.7769 - val_loss: 4.7769
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - categorical_accuracy: 0.4119 - categorical